In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys
from pathlib import Path

sys.path.append(str(Path().cwd().parent))

In [57]:
from typing import Tuple

from itertools import product

import pandas as pd

from plotting import plot_ts
from load_dataset import Dataset
from model import TimeSeriesPredictor

### Какие ряды будем тестировать?

* длинный ряд с сезонностью  
* короткий ряд с сезонностью  
* короткий ряд с сезонностью и трендом  
* случайное блуждание  
* средне зашумленный ряд
* "шумный" ряд  

In [4]:
ds = Dataset('../data/dataset/')

In [5]:
long = ds['daily-min-temperatures.csv']

In [6]:
plot_ts(long)

Loading BokehJS ...

In [7]:
short_season = ds['hour_3019.csv'][300:]

In [8]:
plot_ts(short_season)

Loading BokehJS ...

In [9]:
short_season_trend = ds['international-airline-passengers.csv']

In [10]:
plot_ts(short_season_trend)

Loading BokehJS ...

In [11]:
random_walk = ds['dow_jones_0.csv']

In [12]:
plot_ts(random_walk)

Loading BokehJS ...

In [13]:
medium_noize = ds['hour_3426.csv'][300:]

In [14]:
plot_ts(medium_noize)

Loading BokehJS ...

In [15]:
full_noize = ds['day_1574.csv']

In [16]:
plot_ts(full_noize)

Loading BokehJS ...

### Какие модели будем тестировать?

* скользящее среднее
* экспоненциальное сглаживание
* autoArima
* линейная регрессия
* линейная регрессия с L1 регуляризацией (Ridge)
* RandomForeset
* градиентный бустинг


In [27]:
from estimators import RollingEstimator, ExponentialSmoothingEstimator
from pmdarima import auto_arima
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

### По каким метрикам будем сравнивать?

* mse
* mae
* R2
* mase

In [28]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score

from metrics import mean_absolute_percentage_error as mape
from metrics import mase

### По какой методике будем тестировать?

* 70% трейн, 30% тест
* Out-of-sample, чтобы посмотреть как модель предсказывает "вдолгую"
* In-Sample, чтобы посмотреть как модель предсказывает на одну точку вперед
* Для поиска гиперпараметров можно делать кроссвалидацию на тесте по метрике mse

### Задание 1. Напишите функцию, разбивающую на train и test

In [30]:
def train_test_split(ts: pd.Series, ratio: float = 0.7) -> Tuple[pd.Series]:
    split_idx = int(len(ts) * ratio)
    ts_train, ts_test = ts[:split_idx], ts[split_idx:]
    # ваш код здесь
    return ts_train, ts_test

### Зададим соответствие гранулярностей для наших рядов.

In [31]:
granularity_mapping = {
    'long': 'P1D',
    'short_season': 'PT1H',
    'short_season_trend': 'P1M',
    'random_walk': 'P1D',
    'medium_noize': 'PT1H',
    'full_noize': 'P1D'
}

### Задание 2. Напишите функцию, имплементирующую весь пайплайн обучения и прогноза через TimeSeriesPredictor.

* принмает на вход исходный ряд, гранулярность, количество лагов, модель, а также **kwargs, в которые мы будем передавать параметры модели

* разбивает ряд на train/test

* создает инстанс TimeSeriesPredictor с нужными параметрами

* обучает предиктор на трейне

* делает out_of_sample и in_sample прогноз

* возвращает train, test, in_sample, out_of_sample

In [74]:
def make_pipeline(ts: pd.Series, granularity: str, model: callable, **kwargs) -> Tuple[pd.Series]: 
    train, test = train_test_split(ts)
    predictor = TimeSeriesPredictor(
        granularity=granularity,
        num_lags=kwargs['num_lags'],
        model=model,
    )
    
    predictor.set_params(**kwargs)
    predictor.fit(train)
    
    in_sample = predictor.predict_batch(train, test)
    out_of_sample = predictor.predict_next(train, len(test))
    
    return train, test, in_sample, out_of_sample

In [33]:
def hyperparameters_search(ts, model, param_grid, verbose=False):
    
    statistics_in_sample, statistics_out_of_sample = {}, {}
    
    for param_tuple in product(*param_grid.values()):
        params = dict(zip(param_grid.keys(), param_tuple))
        train, test, in_sample, out_of_sample = make_pipeline(
            ts, 'P1D', model, **params)
        
        mse_in_sample = mse(test, in_sample)
        mae_in_sample = mae(test, in_sample)
        r2_score_in_sample = r2_score(test, in_sample)
        mase_in_sample = mase(in_sample, test)
        
        mse_out_of_sample = mse(test, out_of_sample)
        mae_out_of_sample = mae(test, out_of_sample)
        r2_score_out_of_sample = r2_score(test, out_of_sample)
        mase_out_of_sample = mase(out_of_sample, test)
        
        statistics_in_sample[param_tuple] = mse_in_sample
        statistics_out_of_sample[param_tuple] = mse_out_of_sample
        

        if verbose:
            print(f'Params are: {params}')
            print(
                f"""
                IN_SAMPLE
                mse: {mse(test, in_sample)},
                mae: {mae(test, in_sample)},
                r2: {r2_score(test, in_sample)},
                mase: {mase(in_sample, test)}
                ---------------------
                OUT_OF_SAMPLE
                mse: {mse(test, out_of_sample)},
                mae: {mae(test, out_of_sample)},
                r2: {r2_score(test, out_of_sample)},
                mase: {mase(out_of_sample, test)}
                """
            )
            plot_ts(train, test, in_sample, out_of_sample)
            
    
    best_in_sample = sorted(statistics_in_sample.items(), key=lambda x: x[1])[0]
    best_out_of_sample = sorted(statistics_out_of_sample.items(), key=lambda x: x[1])[0]
    
    best_in_sample = dict(zip(param_grid.keys(), best_in_sample[0]))
    best_out_of_sample = dict(zip(param_grid.keys(), best_out_of_sample[0]))
    
    
    if verbose:
        print(f"best in_sample params are {in_sample}")
        print(f"best out_of_sample params are {out_of_sample}")
    
    return best_in_sample, best_out_of_sample

### Задание 3. Напишите функцию, имплементирующую весь пайплайн обучения и прогноза через auto_arima

* функция должна принимать исходный временной ряд, период сезонности, параметры дифференцирования d, D и boolean параметр seasonal, данные параметры будут являться для нас гиперпараметрами, все остальное за нас должна найти auto_arima

* разбивает на train, test

* обучает arima на train при помощи вызова функции auto_arima из библиотеки pmdarima с переданными параметрами и со следующими зафиксированными параметрами: `max_p=3, max_q=3, trace=True, error_action='ignore', suppress_warnings=True, stepwise=True`

* в качестве out_of_sample прогноза просто вызовите метод predict

* в качестве in_sample прогноза обучите модель заново на всём ряде методом `fit`, вызовите метод predict_in_sample и в качестве прогноза возьмите `in_sample_predictions(-len(test):)`

* возвращает train, test, in_sample, out_of_sample (не забудьте сделать их pd.Series с нужным индексом!!)

In [34]:
def make_pipeline_arima(ts: pd.Series, period: int, d: int = 1, D: int = 1, seasonal: bool = True) -> Tuple[pd.Series]:
    train, test = train_test_split(ts)
    
    arima_fit = auto_arima(
        train,
        max_p=3, max_q=3, m=period,
        seasonal=seasonal,
        d=d, D=D,
        trace=True,
        error_action='ignore',
        suppress_warnings=True,
        stepwise=True
    )
    
    out_of_sample = pd.Series(data=arima_fit.predict(len(test)), index=test.index)
    
    arima_fit.fit(ts)
    
    in_sample = pd.Series(arima_fit.predict_in_sample()[-len(test):], index=test.index)
    
    return train, test, in_sample, out_of_sample

### Задание 4. "Прогоните" все алгоритмы на всех рядах и получите сводную таблицу результатов по всем метрикам, постройте также графики прогнозов. 

### short_season

In [111]:
plot_ts(medium_noize)

Loading BokehJS ...

In [122]:
train, test, in_sample, out_of_sample = make_pipeline_arima(full_noize, period=24, seasonal=False, d=0, D=0)

/Users/gsinyakov/ts_spring/.env/lib/python3.8/site-packages/pmdarima/arima/_validation.py:62: UserWarning: m (24) set for non-seasonal fit. Setting to 0
  warnings.warn("m (%i) set for non-seasonal fit. Setting to 0" % m)


Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=-439.119, Time=0.16 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=-230.626, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=-406.103, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=-286.023, Time=0.04 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=-430.402, Time=0.05 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=-422.734, Time=0.07 sec
 ARIMA(3,0,2)(0,0,0)[0]             : AIC=-422.549, Time=0.19 sec
 ARIMA(2,0,3)(0,0,0)[0]             : AIC=-439.591, Time=0.14 sec
 ARIMA(1,0,3)(0,0,0)[0]             : AIC=-434.087, Time=0.12 sec
 ARIMA(3,0,3)(0,0,0)[0]             : AIC=-438.952, Time=0.23 sec
 ARIMA(2,0,3)(0,0,0)[0] intercept   : AIC=-449.322, Time=0.12 sec
 ARIMA(1,0,3)(0,0,0)[0] intercept   : AIC=-451.204, Time=0.11 sec
 ARIMA(0,0,3)(0,0,0)[0] intercept   : AIC=-454.075, Time=0.12 sec
 ARIMA(0,0,2)(0,0,0)[0] intercept   : AIC=-447.249, Time=0.09 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept

In [123]:
plot_ts(train, test, in_sample, out_of_sample)

Loading BokehJS ...

In [119]:
plot_ts(*make_pipeline(full_noize, 'P1D', GradientBoostingRegressor, num_lags=48))

Loading BokehJS ...

In [58]:
param_grid = {
    'model__max_depth': [6, 12],
    'model__n_estimators': [50, 500, 1000],
    'num_lags': [24, 48]
}

best_in_sample, best_out_of_sample = hyperparameters_search(
                                        short_season,
                                        RandomForestRegressor,
                                        param_grid,
                                        verbose=False
                                    )

train, test, in_sample, _ = make_pipeline(
    short_season, 'PT1H', RandomForestRegressor, **best_in_sample)

train, test, _, out_of_sample = make_pipeline(
    short_season, 'PT1H', RandomForestRegressor, **best_out_of_sample)

In [59]:
plot_ts(train, test, in_sample, out_of_sample)

Loading BokehJS ...

In [61]:
train, test, in_sample, out_of_sample = make_pipeline(long, 'P1D', RandomForestRegressor, num_lags=365)

In [62]:
plot_ts(train, test, in_sample, out_of_sample)

Loading BokehJS ...

### Выводы

Ряды с короткой историей.

* на рядах с короткой историей нет смысла использовать "деревянные" алгоритмы, вследствие трудностей с переобучением
* линейную регрессию можно использовать для прогноза на одну точку, нужно быть аккуратным для прогноза на много лагов
* хорошо подходит арима (нужно подбирать d, D); если у вас сложный кейс (полиномиальный тренд, мультипликативная сезонность) - настраивать ариму стоит вручную


Ряды с трендом.
* нельзя использовать "деревянные алгоритмы", т.к. они не умеют улавливать тренд
* линейная регрессия может быть использовать для прогноза на одну точку
* хорошо подходит arima

Интегрированные ряды (блуждания, акции и тп).
* либо наивное
* либо arima (которая выродится в наивное)

Зашумленные ряды.
* главное - не переобучить модель
* arima из коробки работает "почти хорошо"



Таким образом, arima почти везде показывает хорошо, однако у нее есть две проблемы:
* невозомжность автоматически сделать ряд стационарным
* нелинейный рост сложности алгоритма с ростом данных => проблемно использовать на длинных рядах или на рядах с большим количеством доп регрессоров (признаков)

Для предсказания на одну точку вперед можно построить хорошую модель гораздо проще, в частности линейная регрессия почти везде показывает себя хорошо. 


Деревянные модели показывают себя хорошо только на рядах с большим количеством данных.